# Steps to Tackle a Time Series Problem (with Codes in Python)
Note: These are just the codes from article

## Loading and Handling TS in Pandas

In [77]:
import pandas as pd
import warnings
import xlrd
import sys
from pandas.plotting import register_matplotlib_converters
import numpy as np
from datetime import datetime,timedelta
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as plt
import seaborn as sea
from sklearn.linear_model import LassoCV, RidgeCV, LinearRegression

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,12
rcParams['font.sans-serif']=['SimHei']
rcParams['axes.unicode_minus'] = False
warnings.filterwarnings(action='once')

In [78]:
#data_history是对象历史负荷表
data_history = pd.read_csv('tq_past_load.csv',encoding="gb18030",header=0,parse_dates=['2017/6/8','2017/6/15','2017/6/22','2017/6/29','2017/7/6','2017/7/13','2017/7/20','2017/7/27','2017/8/3','2017/8/10','2017/8/17','2017/8/24','2017/8/31','2017/9/7','2017/9/14','2017/9/21','2017/9/28','2017/10/5','2017/10/12','2017/10/19','2017/10/26','2017/11/2','2017/11/9','2017/11/16','2017/11/23','2017/11/30','2017/12/7','2017/12/14','2017/12/21','2017/12/28','2018/1/4','2018/1/11','2018/1/18','2018/1/25','2018/2/1','2018/2/8','2018/2/15','2018/2/22','2018/3/1','2018/3/8','2018/3/15','2018/3/22','2018/3/29','2018/4/5','2018/4/12','2018/4/19','2018/4/26','2018/5/3','2018/5/10','2018/5/17','2018/5/24','2018/5/31','2018/6/7','2018/6/14','2018/6/21','2018/6/28','2018/7/5','2018/7/12','2018/7/19','2018/7/26','2018/8/2','2018/8/9','2018/8/16','2018/8/23','2018/8/30','2018/9/6','2018/9/13','2018/9/20','2018/9/27','2018/10/4','2018/10/11','2018/10/18','2018/10/25','2018/11/1','2018/11/8','2018/11/15','2018/11/22','2018/11/29','2018/12/6','2018/12/13','2018/12/20','2018/12/27','2019/1/3','2019/1/10','2019/1/17','2019/1/24','2019/1/31','2019/2/7','2019/2/14','2019/2/21','2019/2/28','2019/3/7','2019/3/14','2019/3/21','2019/3/28','2019/4/4','2019/4/11','2019/4/18','2019/4/25','2019/5/2','2019/5/9','2019/5/16','2019/5/23'])

C:\ProgramData\Anaconda3\envs\p36workshop\lib\site-packages\xlrd\xlsx.py:266: PendingDeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\ProgramData\Anaconda3\envs\p36workshop\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
data_history.bs = data_history.bs.apply(lambda x:x[1:])
data_history = data_history.set_index('bs')


In [89]:
#连接每个对象的历史负荷
data_sort_history = data_cluster_industry.join(data_history,rsuffix='r_')

In [90]:
#提取立户日期中的年份
data_sort_history[["lhrq"]] = data_sort_history[["lhrq"]].astype(str)
data_sort_history.lhrq = data_sort_history.lhrq.apply(lambda x:x[:4])

In [91]:
#删除无关列
data_sort_history.drop(columns=['tqlx','tqlxr_','yxztdm','edrl','yhbh','yhmc','week_day_year200','day80','rowid'],inplace=True)

In [92]:
data_sort_history.dropna(subset=['ydlbdm','industry_type'],inplace=True)

In [ ]:
data_draw = data.drop(columns=['相似度\n分类代码','聚类\n代码','聚类\n平均误差','台区名称','用户名称','lhrq','学校类型','age'])

data_draw.set_index('bs',inplace=True)
data_draw = data_draw.T
print(data_draw.columns)
print(data_draw.shape[1])

for i in range(data_draw.shape[1]-1):
    for j in range(data_draw.shape[0]-1):
        try:
            if data_draw.iloc[j,i] is None:
                data_draw.iloc[j,i] = (data_draw.iloc[j+1,i]+data_draw.iloc[j-1,i])/2
        except IndexError:
            print(j,i)

In [ ]:
register_matplotlib_converters()
plt.figure(figsize=(38,36), dpi=80)
for i in range(data_draw.shape[1]):
    plt.plot(data_draw.iloc[:,i],label=data_draw.columns[i],linewidth=i)
leg = plt.legend(loc='best')
plt.savefig('data_draw.png')
plt.show()

In [ ]:
data_increase_draw = data_increase.drop(columns=['相似度\n分类代码','聚类\n代码','聚类\n平均误差','台区名称','用户名称','lhrq','学校类型','age'])

data_increase_draw.set_index('bs',inplace=True)
data_increase_draw = data_increase_draw.T


In [ ]:
register_matplotlib_converters()
plt.figure(figsize=(38,36), dpi=80)
for i in range(data_draw.shape[1]):
    plt.plot(data_increase_draw.iloc[:,i],label=data_increase_draw.columns[i],linewidth=i)
leg = plt.legend(loc='best')
plt.savefig('data_increase_draw.png')
plt.show()

In [ ]:
# 删除含有负值和零值的行
# datatemp=data.drop(columns=['tqlx','tqmc','bs'])
# droplist=[]
# for i in range(len(datatemp.index)):
#     if (datatemp.iloc[i].values<=np.array(0)).any():
#         droplist.append(datatemp.iloc[i].name)
# data=data.drop(labels=droplist)
# data.head()

In [ ]:
# datatemp=data.drop(columns=['tqlx','tqmc','bs'])
# print((datatemp.index.isin([0])).size)
# datatemp[datatemp.index.isin([0])]

In [ ]:
#每一行的空值用每一行的平均值来填充
# for row in data.index:
#     data.loc[row].fillna(data.loc[row].drop(['tqlx','tqmc','bs']).mean(),inplace=True)
# data.head()

In [ ]:
#tqlx和tqmc在时间序列分析中暂时用不到，删除
# data.tqmc.str.contains('学校|大学|小学|中学')
# data.loc[data.tqmc.str.contains('学校|大学|小学|中学',na=False)]
import copy
data_time_series = copy.deepcopy(data.drop(columns=['相似度\n分类代码','聚类\n代码','聚类\n平均误差','台区名称','用户名称']))
data_time_series.reset_index(inplace=True)
data_time_series.head()

In [ ]:
#转置
data_time_series=data_time_series.T
#把标识行设为列名
data_time_series.columns = data_time_series.iloc[0]
data_time_series.drop(index='bs',inplace=True)
data_time_series.head()

In [ ]:
#将索引变为列，方便之后格式转换
data_time_series.index.names=['time_series']
data_time_series=data_time_series.reset_index()
data_time_series.head()

Reading as datetime format:

In [ ]:
#将含有时序数据的字段转化为datetime64格式
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y/%m/%d')
# dateparse('2017-08-01')
data_time_series['time_series'] = data_time_series['time_series'].apply(dateparse)
data_time_series.set_index('time_series',inplace=True)
data_time_series=data_time_series.apply(pd.to_numeric)
data_time_series.head()

In [ ]:
#处理空值
# data_time_series=data_time_series.fillna(method='backfill',axis='index')
# data_time_series_dropna=data_time_series.dropna(axis='columns')
#去掉重复的台区（重复列）
# data_time_series_dropna=data_time_series_dropna.T.drop_duplicates().T
# data_time_series_dropna.index

In [ ]:
#粗筛出原始数据缺失值比较少的台区（其表现为填充空值操作后连续重复值较少）,保存为一个列表
# drop_little_tq_index = data_time_series_dropna.T\
# [data_time_series_dropna.T['2017-06-08']!=data_time_series_dropna.T['2017-08-03']].index.to_list()

In [ ]:
# print(data_time_series_dropna.columns.is_unique)
#打印列的重复次数
# List=data_time_series_dropna.columns.to_list()
# List_set = set(List) 
# for item in List_set:
#     if List.count(item)>1:
#         print("the %s has found %d" %(item,List.count(item)))

In [ ]:
# data_2017_max=pd.DataFrame(data_time_series_dropna['2017'].max(),columns=['2017'])
# data_2018_max=pd.DataFrame(data_time_series_dropna['2018'].max(),columns=['2018'])
# data_2019_max=pd.DataFrame(data_time_series_dropna['2019'].max(),columns=['2019'])

# print(data_2017_max.head())

# print(type(data_2017_max))
# print(data_2017_max.head())
# data_2017_max.index.is_unique

In [ ]:
# bs_year_max=pd.concat([data_2017_max,data_2018_max,data_2019_max],axis=1,join='inner')
# bs_year_max.index.is_unique

In [ ]:
#粗筛出逐年增长的负荷数据
# increase_tq_index = bs_year_max[(bs_year_max['2017']<bs_year_max['2018']) & (bs_year_max['2018']<bs_year_max['2019'])].index.to_list()

In [ ]:
#打印出既是逐年增长也没有太多缺失值的台区
# intersection_index = list(set(drop_little_tq_index).intersection(set(increase_tq_index)))
# print(intersection_index)

In [ ]:
#convert to time series:
ts = data_time_series_dropna[3002232277]
ts.head()

# 平稳性检验

## 展示时序图

In [ ]:
register_matplotlib_converters()
plt.plot(ts)

### 平稳性检验函数

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #移动平均和移动方差
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    #把移动平均和移动方差画出来:
    orig = plt.plot(timeseries, color='blue',label='原始曲线')
    mean = plt.plot(rolmean, color='yellow', label='移动平均')
    std = plt.plot(rolstd, color='black', label = '移动方差')
    plt.legend(loc='best')
    plt.title(u'绘制移动平均数和移动方差，观察它是否随着时间变化')
    plt.show(block=False)
    
    #进行单位根检验:
    print('单位根检验:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    if dfoutput.loc['p-value']>0.01:
        print('p值大于0.01，不能拒绝原假设，说明时间序列是非平稳的')
    else:
        print('p值小于0.01，拒绝原假设，说明时间序列是平稳的')
    return dfoutput.loc['p-value']<0.01
    #返回时间序列是否平稳的判断值
    '''
    单位根的原假设为序列具有单位根，即非平稳，
    对于一个平稳的时序数据，就需要在给定的置信水平上显著，拒绝原假设。
    '''

In [ ]:
test_stationarity(ts)

# 平稳性的处理


## 预测和消除趋势


## Smoothing:

### 移动平均

In [ ]:
moving_avg = ts.rolling(12).mean()
plt.plot(ts,label='原始曲线')
plt.plot(moving_avg, color='red',label='移动平均')
plt.title('移动平均')
plt.legend(loc='best')
plt.show()

In [ ]:
ts_moving_avg_diff = ts - moving_avg
ts_moving_avg_diff.dropna(inplace=True)
test_stationarity(ts_moving_avg_diff)

### 指数平滑法

In [ ]:
# expwighted_avg = ts.ewm(halflife=12).mean()
# plt.plot(ts,label='原始曲线')
# plt.plot(expwighted_avg, color='red',label='指数平滑曲线')
# plt.title('指数平滑处理')
# plt.legend(loc='best')
# plt.show()
#expwighted_avg.plot(style='k--')

In [ ]:
# ts_ewma_diff = ts - expwighted_avg
# test_stationarity(ts_ewma_diff)

## 消除趋势和季节性

### 差分:

In [ ]:
#直接对原始数据取一阶差分:
ts_diff = ts - ts.shift(1)
plt.plot(ts_diff)
plt.title('差分处理')
plt.show()

In [ ]:
ts_diff.dropna(inplace=True)
test_stationarity(ts_diff)

# 构建模型

## 时间序列的分解（分解后分开预测）

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts,extrapolate_trend='freq')
# 显式调用seasonal_decompose函数的extrapolate_trend参数可以强制提取残差，同时使趋势曲线更加平滑
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ts, label='原始')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='趋势')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='季节')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='残差')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
trend_diff = trend - trend.shift(1)
plt.plot(trend_diff)
plt.title('趋势的差分处理')
plt.show()

In [ ]:
trend_diff.dropna(inplace=True)
test_stationarity(trend_diff)

In [ ]:
ts_decompose = residual
ts_decompose.dropna(inplace=True)
test_stationarity(ts_decompose)

## 利用交叉验证法求预测模型参数

# 创建训练集和测试集
size = int(len(ts_diff) * 0.8)
train, test = ts_diff[0:size], ts_diff[size:len(ts_diff)]
register_matplotlib_converters()
plt.plot(train.index, train, color='blue', label='训练集')
plt.plot(test.index, test, color='green', label='测试集')
plt.legend()
plt.title('真实数据的训练集和测试集')
plt.show()

#样本内预测的结果和预测误差
def compare_ARIMA_modes_testing(order):
    history = [x for x in train]
    predictions_f = list()
    predictions_p = list()
    for t in range(len(test)):
        model = ARIMA(history, order=order)
        model_fit = model.fit(disp=-1)
        yhat_f = model_fit.forecast()[0][0]
        yhat_p = model_fit.predict(start=len(history), end=len(history))[0]
        predictions_f.append(yhat_f)
        predictions_p.append(yhat_p)
        history.append(test[t])
    #mean_squared_error即为“误差”的平方的期望值
    error_f = mean_squared_error(test, predictions_f)
    error_p = mean_squared_error(test, predictions_p)
    print('MSE forecast:\t\t\t{:1.4f}'.format(error_f))
    print('MSE predict:\t\t\t{:1.4f}'.format(error_p))
    return {'Predictions forecast': pd.Series(predictions_f,index=test.index),
            'Predictions predict': pd.Series(predictions_p,index=test.index),
            'MSE forecast': error_f,
            'MSE predict': error_p}

#利用交叉验证法寻找最佳阶数
def cross_proper_model(maxLag):
    init_error = sys.maxsize
    init_p = 0
    init_q = 0
    init_properModel = None
    for p in np.arange(maxLag):
        for q in np.arange(maxLag):
            arima = compare_ARIMA_modes_testing((p,1,q))
            error_f = arima['MSE forecast']
            error_p = arima['MSE predict']
        if error_f < init_error or error_p < init_error:
                init_p = p
                init_q = q
                init_properModel =arima
                init_error = min(error_f,error_p)
    return [init_error, init_p, init_q, init_properModel]

## 利用贝叶斯信息准则求预测模型参数

In [ ]:
#利用贝叶斯信息准则（BIC）寻找最佳阶数，使得BIC达到最小值的(p, q)即为该准则下的最优模型的阶数
def proper_model(data_ts, maxLag):
    init_bic = sys.maxsize
    init_p = 0
    init_q = 0
    init_properModel = None
    for p in np.arange(maxLag):
        for q in np.arange(maxLag):
            model = ARMA(data_ts, order=(p, q),freq=data_ts.index.inferred_freq)
            try:
                results_ARMA = model.fit(disp=-1, method='css')
            except:
                continue#忽略所有异常
            bic = results_ARMA.bic
            if bic < init_bic:
                init_p = p
                init_q = q
                init_properModel = results_ARMA
                init_bic = bic
    return [init_bic, init_p, init_q, init_properModel]

## 预测模型（针对趋势和残差）

In [ ]:
#样本外预测
def compare_ARIMA_modes(series,order):
    history_f = [x for x in series]
    history_p = [x for x in series]
    series_p = series
    series_f = series
#     for t in range(52):滚动预测的效果不佳
    model = ARIMA(history_f, order=order)
    model_fit = model.fit(disp=-1)
    yhat_f = model_fit.forecast(steps=104)[0]
    model = ARIMA(history_p, order=order)
    model_fit = model.fit(disp=-1)
    yhat_p = model_fit.predict(start=len(history_p), end=len(history_p)+104)
    
    #附加新元素时也加上一个单位的索引
    for t in range(104,1,-1):
        series_p = series_p.append(pd.Series(yhat_p[-t],index=[series_p.index[-1]+timedelta(days=7)]))
        series_f = series_f.append(pd.Series(yhat_f[-t],index=[series_f.index[-1]+timedelta(days=7)]))
    return {'Predictions forecast': series_f[series.index[-1]:series_f.index[-1]],
            'Predictions predict': series_p[series.index[-1]:series_p.index[-1]]}

In [ ]:
#样本外预测
def LinearRegression_modes(series):
    history = [x for x in series]
    series_p = series
    #lr = LassoCV(cv=tscv)
    lr = LinearRegression(normalize= "l1") 
    #可以尝试随机森林的效果，也不错。也可以做多模型结果融合，请自己尝试。
    #lr = RandomForestRegressor(n_estimators=100, max_depth=10) #lag_start = 288, lag_end = 320
    # lr = RidgeCV(cv = tscv)
    lr.fit(np.array(range(len(series.index))).reshape(1, -1), np.array(series.values).reshape(1, -1))
    #train_score = lr.score(x_train_scaled, y_train)
    #test_score = lr.score(x_test_scaled, y_test)
    #print("num_tree", each, "score", train_score, test_score)
    #print(x_pred)
    # now 拟合
    print(np.array(range(len(series.index))).reshape(1, -1))
    y_fit = lr.predict(np.array(range(len(series.index),len(series.index)+103)).reshape(1, -1))
    print(y_fit[0])
    print(series.values)
    #附加新元素时也加上一个单位的索引
    for t in range(52,1,-1):
        series_p = series_p.append(pd.Series(y_fit[0][-t],index=[series_p.index[-1]+timedelta(days=7)]))
    return {'Predictions': series_p[series.index[-1]:series_p.index[-1]]}

In [ ]:
#季节变动每一年都是一样的，预测季节因素只需要将前一年的数据复制到新一年即可
def season_forcast_year(seasonal):
    seasonal_f = seasonal
    seasonal_f.index = seasonal.index+timedelta(weeks=104)
    return seasonal_f

In [ ]:
linear_predict = LinearRegression_modes(trend)

In [ ]:
print(linear_predict['Predictions'].values)

In [ ]:
# type(y_fit)

In [ ]:
# trend_coefficient=proper_model(trend_diff,20)
# print(trend_coefficient)

In [ ]:
# trend_model =  compare_ARIMA_modes(trend_diff,(trend_coefficient[1], 1, trend_coefficient[2]))

In [ ]:
# forecast and predict are different for ARMA
# print()
# plt.plot(trend, label='Ground Truth')
# plt.plot(linear_predict['Predictions'][1:-1]+2.5, color='red', label='.forecast()',linestyle='dashed')
# plt.legend(loc='best')
# plt.title('ARMA')
# plt.show()

In [ ]:
trend_model =  compare_ARIMA_modes(trend,(0, 1, 0))

In [ ]:
# forecast and predict are different for ARMA
print()
plt.plot(trend, label='Ground Truth')
plt.plot(linear_predict['Predictions'], color='red', label='.forecast()',linestyle='dashed')
plt.legend(loc='best')
plt.title('ARMA')
plt.show()

In [ ]:
# predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()

In [ ]:
# residual_coefficient=proper_model(residual,20)
# print(residual_coefficient)

In [ ]:
residual_model =  compare_ARIMA_modes(residual,(18, 1, 8))

In [ ]:
plt.plot(residual, label='Ground Truth')
plt.plot(residual_model['Predictions forecast'], color='red', label='.forecast()',linestyle='dashed')
plt.plot(residual_model['Predictions predict'], color='green', label='.predict()',linestyle='dotted')
plt.legend(loc='best')
plt.title('residual_model')
plt.show(block=False)

In [ ]:
seasonal_f=season_forcast_year(seasonal)

In [ ]:
forcast_model = linear_predict['Predictions'][1:-1]+2.5+residual_model['Predictions forecast']+seasonal_f
forcast_model.fillna(11.578118,inplace=True)
forcast_model.head()

In [ ]:
forcast_model_least=linear_predict['Predictions'][1:-1]+2.5+seasonal_f
#forcast_model_least.fillna(11.578118,inplace=True)
forcast_model.head()

# 预测

In [ ]:
import math
ts_past_list = (ts.index-timedelta(weeks=103)).to_list()
ts_later_list = (ts.index+timedelta(weeks=103)).to_list()
ts_list = ts.index.to_list()
ts_list=ts_past_list+ts_list+ts_later_list
series=pd.Series(np.arange(0, 12.36, 0.04),index=ts_list)
plt.plot(series,label='未处理')
plt.legend(loc='best')
for i in [5,10,30,50]:
    series_p=series.apply(lambda x:10/(1+i*math.e**(1-x))+1.5)
    plt.plot(series_p,label='i为'+str(i))
    plt.legend(loc='best')

In [ ]:
print(type(ts.index))

In [ ]:
plt.plot(series_p.loc[pd.concat([ts.index.to_series(),forcast_model.index.to_series()]).index],label='S型曲线')
plt.plot(ts, label='历史负荷')
plt.plot(forcast_model, color='red', label='预测曲线上界')
plt.plot(forcast_model_least, color='green', label='预测曲线下界',linestyle='dashed')
plt.legend(loc='best')
plt.title('预测结果')
plt.savefig('foo.png')
plt.show()

In [ ]:
plt.plot(forcast_model, color='red', label='预测曲线上界')
plt.plot(forcast_model_least, color='green', label='预测曲线下界',linestyle='dashed')
plt.legend(loc='best')
plt.title('预测结果')
plt.savefig('foo.png')
plt.show()

In [ ]:
plt.plot(ts, label='Ground Truth')

plt.legend(loc='best')
plt.title('forcast_model')
plt.show()

In [ ]:
plt.plot(ts[-52:-1])

In [ ]:
plt.plot(ts[-104:-53])

In [ ]:
plt.plot(forcast_model)

# 预测时间序列

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ts_diff, lags=40, ax=ax1) #从13开始是因为做季节性差分时window是12
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ts_diff, lags=40, ax=ax2)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(trend.dropna(), lags=40, ax=ax1) #从13开始是因为做季节性差分时window是12
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(trend.dropna(), lags=40, ax=ax2)

### ACF & PACF Plots

In [ ]:
#选择差分处理后的结果进行预测
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf  

lag_acf = acf(ts_diff, nlags=20)
lag_pacf = pacf(ts_diff, nlags=20, method='ols')

#Plot ACF:    
plt.subplot(121)    
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

#两条虚线之间是置信区间

In [ ]:
#选择季节分解后的趋势进行预测
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf  

lag_acf = acf(trend.dropna(), nlags=20)
lag_pacf = pacf(trend.dropna(), nlags=20, method='ols')

#Plot ACF:    
plt.subplot(121)    
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(trend.dropna())),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(trend.dropna())),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(trend.dropna())),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(trend.dropna())),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

#两条虚线之间是置信区间

### AR Model:

In [ ]:
#MA model:
model = ARIMA(ts_log, order=(1, 1, 2),freq=ts_log.index.inferred_freq)  
results_AR = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-ts_log_diff)**2))

### MA Model

In [ ]:
model = ARIMA(ts_log, order=(0, 1, 2),freq=ts_log.index.inferred_freq)  
results_MA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-ts_log_diff)**2))

### ARIMA Model:

In [ ]:
model = ARIMA(ts_log, order=(2, 1, 2),freq=ts_log.index.inferred_freq)  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

### 预测还原:

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print(predictions_ARIMA_diff.head())

In [ ]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum.head())

In [ ]:
predictions_ARIMA_log = pd.Series(ts_log.iloc[0], index=ts_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
plt.plot(ts_log)
plt.plot(predictions_ARIMA_diff_cumsum)

In [ ]:
plt.plot(ts_log)
plt.plot(predictions_ARIMA_log)
predictions_ARIMA_log=predictions_ARIMA_log.rename('predictions_ARIMA_log')
predictions_ARIMA_log.to_csv(predictions_ARIMA_log.name+'.csv',header=True)

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))

### 对分解出来的趋势进行预测